In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from scipy.spatial.distance import cityblock, euclidean

In [15]:
data = pd.read_csv("kaggle/DSL-StrongPasswordData.csv")
subjects = data["subject"].unique()


In [16]:
print(data.isna().sum())


subject            0
sessionIndex       0
rep                0
H.period           0
DD.period.t        0
UD.period.t        0
H.t                0
DD.t.i             0
UD.t.i             0
H.i                0
DD.i.e             0
UD.i.e             0
H.e                0
DD.e.five          0
UD.e.five          0
H.five             0
DD.five.Shift.r    0
UD.five.Shift.r    0
H.Shift.r          0
DD.Shift.r.o       0
UD.Shift.r.o       0
H.o                0
DD.o.a             0
UD.o.a             0
H.a                0
DD.a.n             0
UD.a.n             0
H.n                0
DD.n.l             0
UD.n.l             0
H.l                0
DD.l.Return        0
UD.l.Return        0
H.Return           0
dtype: int64


In [17]:
data = data.dropna()


In [18]:

def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = (x[0] - x[1]) / (y[1] - x[1] - y[0] + x[0])
    eer = x[0] + a * (y[0] - x[0])
    return eer


In [23]:
class ManhattanFilteredDetector:
    
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.std_vector = []
        self.subjects = subjects
        
    def training(self):
        # Compute mean and std dev vectors for the training data
        self.mean_vector = self.train.mean().values
        self.std_vector = self.train.std().values
        
        dropping_indices = []
        
        # Check each training sample
        for i in range(self.train.shape[0]):
            cur_score = euclidean(self.train.iloc[i].values, self.mean_vector)
            
            # Drop if any feature exceeds 3 times its std dev from the mean
            if (cur_score > 3 * self.std_vector).any():
                dropping_indices.append(i)
        
        # Drop the outliers from the training set
        self.train = self.train.drop(self.train.index[dropping_indices])
        
        # Recompute mean after removing outliers
        self.mean_vector = self.train.mean().values
        # Check after dropping/filling NaN in training and testing sets
        print(f"NaN check in training data: {self.train.isna().sum().sum()}")
        print(f"NaN check in genuine test data: {self.test_genuine.isna().sum().sum()}")
        print(f"NaN check in imposter test data: {self.test_imposter.isna().sum().sum()}")

        
    def testing(self):
        # Compute scores for genuine test set (same user)
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, self.mean_vector)
            self.user_scores.append(cur_score)
        
        # Compute scores for imposters
        for i in range(self.test_imposter.shape[0]):
            cur_score = cityblock(self.test_imposter.iloc[i].values, self.mean_vector)
            self.imposter_scores.append(cur_score)
        print(f"Checking NaN in user_scores: {np.isnan(self.user_scores).any()}")
        print(f"Checking NaN in imposter_scores: {np.isnan(self.imposter_scores).any()}")
        # Check after dropping/filling NaN in training and testing sets
        print(f"NaN check in training data: {self.train.isna().sum().sum()}")
        print(f"NaN check in genuine test data: {self.test_genuine.isna().sum().sum()}")
        print(f"NaN check in imposter test data: {self.test_imposter.isna().sum().sum()}")

    def evaluate(self):
        eers = []
        
        for subject in self.subjects:
            
            self.user_scores = []
            self.imposter_scores = []
                # Remove NaN values from user_scores and imposter_scores
            self.user_scores = [score for score in self.user_scores if not np.isnan(score)]
            self.imposter_scores = [score for score in self.imposter_scores if not np.isnan(score)]

            # Consider current subject as genuine and rest as imposters
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
    
            # genuine user's first 200 time vectors for training
            self.train = genuine_user_data[:200]
    
            # True set (200 records)
            self.test_genuine = genuine_user_data[200:]
    
            # False set (250 records, 5 per imposter, 50 imposters in all)
            self.test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            
            # Train and test for each subject
            self.training()
            self.testing()
    
            # Calculate EER for this subject
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        
        return np.mean(eers), np.std(eers)

In [24]:
mean_eer, std_eer = ManhattanFilteredDetector(subjects).evaluate()
print(f"Mean EER: {mean_eer:.4f}, Std EER: {std_eer:.4f}")

NaN check in training data: 0
NaN check in genuine test data: 0
NaN check in imposter test data: 0
Checking NaN in user_scores: True
Checking NaN in imposter_scores: True
NaN check in training data: 0
NaN check in genuine test data: 0
NaN check in imposter test data: 0


ValueError: Input contains NaN.

In [27]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.utils.validation import assert_all_finite

# Function to check for NaN values
def check_nan_in_scores(user_scores, imposter_scores):
    user_nan = np.any(np.isnan(user_scores))
    imposter_nan = np.any(np.isnan(imposter_scores))
    print(f"NaN in user_scores: {user_nan}")
    print(f"NaN in imposter_scores: {imposter_nan}")
    return user_nan, imposter_nan

# Function to evaluate EER (Equal Error Rate)
def evaluateEER(user_scores, imposter_scores):
    # Check for NaN values in scores
    user_nan, imposter_nan = check_nan_in_scores(user_scores, imposter_scores)
    
    # If any NaNs are found, raise an error
    if user_nan or imposter_nan:
        raise ValueError("Input contains NaN.")

    labels = [0] * len(user_scores) + [1] * len(imposter_scores)
    scores = user_scores + imposter_scores

    # Print the scores and labels for debugging
    print(f"user_scores: {user_scores}")
    print(f"imposter_scores: {imposter_scores}")
    print(f"labels: {labels}")

    # Ensure that scores don't contain NaNs
    assert_all_finite(scores, allow_nan=False, input_name="scores")

    # Compute ROC curve and EER
    fpr, tpr, thresholds = roc_curve(labels, scores)
    print(f"FPR: {fpr}")
    print(f"TPR: {tpr}")
    print(f"Thresholds: {thresholds}")

    missrates = 1 - tpr  # Miss rate = 1 - True Positive Rate (TPR)
    farates = fpr        # False Acceptance Rate (FAR)

    # Calculate the Equal Error Rate (EER)
    eer_threshold = thresholds[np.nanargmin(np.abs(missrates - farates))]
    eer = missrates[np.nanargmin(np.abs(missrates - farates))]
    
    print(f"EER: {eer}, Threshold at EER: {eer_threshold}")
    return eer

# Simulated class for detector evaluation
class ManhattanFilteredDetector:
    def __init__(self, subjects):
        self.subjects = subjects
        self.user_scores = []
        self.imposter_scores = []

    def testing(self):
        # Placeholder for actual testing logic
        # You should replace this with actual logic to populate user_scores and imposter_scores
        self.user_scores = [0.8, 0.7, 0.6]  # Example user scores
        self.imposter_scores = [0.4, 0.35, 0.1]  # Example imposter scores

    def evaluate(self):
        eers = []
        for subject in self.subjects:
            print(f"Evaluating subject {subject}")
            self.testing()  # You may need to update this to suit your test case generation logic

            # Calculate EER for this subject
            eer = evaluateEER(self.user_scores, self.imposter_scores)
            eers.append(eer)

        return np.mean(eers), np.std(eers)

# Example usage
subjects = ['s002', 's055', 's003']  # Example subject list
detector = ManhattanFilteredDetector(subjects)

try:
    mean_eer, std_eer = detector.evaluate()
    print(f"Mean EER: {mean_eer:.4f}, Std EER: {std_eer:.4f}")
except ValueError as e:
    print(e)


Evaluating subject s002
NaN in user_scores: False
NaN in imposter_scores: False
user_scores: [0.8, 0.7, 0.6]
imposter_scores: [0.4, 0.35, 0.1]
labels: [0, 0, 0, 1, 1, 1]
FPR: [0.         0.33333333 1.         1.        ]
TPR: [0. 0. 0. 1.]
Thresholds: [inf 0.8 0.6 0.1]
EER: 1.0, Threshold at EER: 0.6
Evaluating subject s055
NaN in user_scores: False
NaN in imposter_scores: False
user_scores: [0.8, 0.7, 0.6]
imposter_scores: [0.4, 0.35, 0.1]
labels: [0, 0, 0, 1, 1, 1]
FPR: [0.         0.33333333 1.         1.        ]
TPR: [0. 0. 0. 1.]
Thresholds: [inf 0.8 0.6 0.1]
EER: 1.0, Threshold at EER: 0.6
Evaluating subject s003
NaN in user_scores: False
NaN in imposter_scores: False
user_scores: [0.8, 0.7, 0.6]
imposter_scores: [0.4, 0.35, 0.1]
labels: [0, 0, 0, 1, 1, 1]
FPR: [0.         0.33333333 1.         1.        ]
TPR: [0. 0. 0. 1.]
Thresholds: [inf 0.8 0.6 0.1]
EER: 1.0, Threshold at EER: 0.6
Mean EER: 1.0000, Std EER: 0.0000


In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from scipy.spatial.distance import cityblock, mahalanobis, euclidean

data = pd.read_csv("kaggle/DSL-StrongPasswordData.csv")
subjects = data["subject"].unique()

def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = (x[0] - x[1]) / (y[1] - x[1] - y[0] + x[0])
    eer = x[0] + a * (y[0] - x[0])
    return eer

class EuclideanDetector:
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.subjects = subjects

    def training(self, train):
        self.mean_vector = train.mean().values

    def testing(self, test_genuine, test_imposter):
        for i in range(test_genuine.shape[0]):
            cur_score = np.linalg.norm(test_genuine.iloc[i].values - self.mean_vector)
            self.user_scores.append(cur_score)

        for i in range(test_imposter.shape[0]):
            cur_score = np.linalg.norm(test_imposter.iloc[i].values - self.mean_vector)
            self.imposter_scores.append(cur_score)

    def evaluate(self):
        eers = []
        for subject in subjects:
            self.user_scores = []
            self.imposter_scores = []
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
            train = genuine_user_data[:200]
            test_genuine = genuine_user_data[200:]
            test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            self.training(train)
            self.testing(test_genuine, test_imposter)
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        return np.mean(eers), np.std(eers)

class ManhattanDetector:
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.subjects = subjects

    def training(self, train):
        self.mean_vector = train.mean().values

    def testing(self, test_genuine, test_imposter):
        for i in range(test_genuine.shape[0]):
            cur_score = cityblock(test_genuine.iloc[i].values, self.mean_vector)
            self.user_scores.append(cur_score)

        for i in range(test_imposter.shape[0]):
            cur_score = cityblock(test_imposter.iloc[i].values, self.mean_vector)
            self.imposter_scores.append(cur_score)

    def evaluate(self):
        eers = []
        for subject in subjects:
            self.user_scores = []
            self.imposter_scores = []
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
            train = genuine_user_data[:200]
            test_genuine = genuine_user_data[200:]
            test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            self.training(train)
            self.testing(test_genuine, test_imposter)
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        return np.mean(eers), np.std(eers)

class MahalanobisDetector:
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.covinv = None
        self.subjects = subjects

    def training(self, train):
        self.mean_vector = train.mean().values
        self.covinv = np.linalg.inv(np.cov(train.T))

    def testing(self, test_genuine, test_imposter):
        for i in range(test_genuine.shape[0]):
            diff = test_genuine.iloc[i].values - self.mean_vector
            cur_score = np.dot(np.dot(diff.T, self.covinv), diff)
            self.user_scores.append(cur_score)

        for i in range(test_imposter.shape[0]):
            diff = test_imposter.iloc[i].values - self.mean_vector
            cur_score = np.dot(np.dot(diff.T, self.covinv), diff)
            self.imposter_scores.append(cur_score)

    def evaluate(self):
        eers = []
        for subject in subjects:
            self.user_scores = []
            self.imposter_scores = []
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
            train = genuine_user_data[:200]
            test_genuine = genuine_user_data[200:]
            test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            self.training(train)
            self.testing(test_genuine, test_imposter)
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        return np.mean(eers), np.std(eers)

# Example usage:
detector = EuclideanDetector(subjects)
mean_eer, std_eer = detector.evaluate()
print(f"EuclideanDetector - Mean EER: {mean_eer}, Std EER: {std_eer}")

detector = ManhattanDetector(subjects)
mean_eer, std_eer = detector.evaluate()
print(f"ManhattanDetector - Mean EER: {mean_eer}, Std EER: {std_eer}")

detector = MahalanobisDetector(subjects)
mean_eer, std_eer = detector.evaluate()
print(f"MahalanobisDetector - Mean EER: {mean_eer}, Std EER: {std_eer}")


EuclideanDetector - Mean EER: 0.16928649036431045, Std EER: 0.09313837450860046
ManhattanDetector - Mean EER: 0.15026997944690795, Std EER: 0.09094241735620097
MahalanobisDetector - Mean EER: 0.27043541471113686, Std EER: 0.14973892469982938


In [35]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cityblock

class ManhattanDetector:
    def __init__(self):
        self.user_scores = []
        self.mean_vector = []

    def training(self, data):
        # Compute the mean vector from the dataset
        self.mean_vector = data.mean().values

    def calculate_threshold(self, data):
        # Calculate distances from the mean vector
        self.user_scores = [cityblock(row.values, self.mean_vector) for _, row in data.iterrows()]
        # Set threshold using percentile; here, 95th percentile is an example
        return np.percentile(self.user_scores, 95)

    def detect_anomalies(self, data, threshold):
        anomalies = []
        for idx, row in data.iterrows():
            score = cityblock(row.values, self.mean_vector)
            if score > threshold:
                anomalies.append(idx)
        return anomalies

# Example usage:

import pandas as pd

try:
    # Use the on_bad_lines parameter to skip or handle problematic lines
    data = pd.read_csv("DSL-StrongPasswordData.csv", delimiter=',', on_bad_lines='skip', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")


detector = ManhattanDetector()
detector.training(data)  # Use the dataset for training

# Calculate threshold for anomaly detection
threshold = detector.calculate_threshold(data)
print(f"Anomaly Detection Threshold: {threshold}")

# Detect anomalies in the same dataset (you can use a different set if available)
anomalies = detector.detect_anomalies(data, threshold)
print(f"Anomalous indices: {anomalies}")


TypeError: Could not convert ['...'] to numeric